In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv("./test_result_vali.csv")
df_valid = pd.read_csv("./validation.csv")

In [3]:
df.shape

(303925, 3)

In [4]:
df_valid.shape

(303925, 25)

In [5]:
df.head()

click  click_pred_per  click_pred_class
0      0        0.000221                 0
1      0        0.000176                 0
2      0        0.000195                 0
3      0        0.000191                 0
4      0        0.000195                 0

In [6]:
pCTR = df['click_pred_per'].as_matrix()

In [7]:
# parameters
# LOW = 10
# HIGH = 40
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df = df_valid
avgCTR = 1793/2430981 #train set
#tuning...
# base_bid = 300

In [8]:
lam = 3.6e-06

In [8]:
# bidprices = base_bid*test_predictions/avgCTR

In [29]:
np.sqrt( pCTR )

array([ 0.02700575,  0.04218362,  0.01303042, ...,  0.0136785 ,
        0.02310326,  0.01690294])

In [10]:
def generate_bidprice(c, lam, pCTR):
#     avgCTR = 1793/2430981#train set
    z = ((pCTR+((c**2)*(lam**2)+(pCTR**2))**(1/2))/(c*lam))**(1/3)
    bidprices = c*(z-1/z)
    assert bidprices.shape == pCTR.shape
    return bidprices

In [11]:
def Prob3(Train_df,payprice,c, lam, pCTR):
        
    current_budget = total_budget
#     win_times = 0
    Train_df['win'] = 0 #initialize win label
    bidprices = generate_bidprice(c, lam, pCTR)
    #start biding...
    for i in Train_df.index:
        if current_budget > 0:
            #generate bidprice:
            current_bidprice = bidprices[i]
            
#             if current_bidprice > current_budget:
#                 print('budget remains:',current_budget)
#                 break
#                 current_bidprice = current_budget
#             bidprices.append(current_bidprice)#record

            #Winning criterion:
            current_payprice = payprice[i]
            if current_bidprice > current_payprice:
                Train_df.at[i, 'win'] = int(1)
                current_budget = current_budget - current_payprice
#                 win_times = win_times + 1
            else:
                Train_df.at[i, 'win'] = int(0)
        else:
            print('lack of budget!',current_budget)
            break
    #biding end        
        
    return Train_df,current_budget

In [30]:
# Train_df,current_budget = Prob3(Train_df,payprice,c, lam, pCTR)

In [31]:
# Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
# Train_df_new = Train_df[Train_df['win']!=0]
# wintimes,_ = Train_df_new.shape
# print('wintimes:',wintimes)
# print('current_budget',current_budget)

# Clk = Train_df_new['click'].sum()
# Imp = Train_df_new['click'].count()
# CTR = Clk/Imp
# Spd = total_budget - current_budget
# CPM = 1000*Spd/Imp
# eCPC = Spd/Clk
    
# Train_Const_summ.loc[0,'Clicks']=Clk
# Train_Const_summ.loc[0,'Spend']=Spd
# Train_Const_summ.loc[0,'CTR']=CTR
# Train_Const_summ.loc[0,'CPM']=CPM
# Train_Const_summ.loc[0,'eCPC']=eCPC

# print(Train_Const_summ)

In [13]:
def calculate_click(Train_df,payprice,c, lam, pCTR):
    print('c:',c,';lambda:',lam)
    result_df,current_budget = Prob3(Train_df,payprice,c, lam, pCTR)
    print('current_budget:',current_budget)
    Train_df_new = result_df[result_df['win']!=0]
    wintimes,_ = Train_df_new.shape
    print('wintimes:',wintimes)
    print('----------------')
    Clk = Train_df_new['click'].sum()    
    return Clk

In [14]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mpld3
import numpy as np

In [16]:
cs = np.arange(0,500,30)
lam = 3.6e-06
# maxClks=[]
# for lam in lams:
Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
fig = plt.figure()
print(max(Clks))
# maxClks.append(max(Clks))
plt.plot(cs,Clks)
mpld3.display(fig)
    
# print('maxmax:',max(maxClks))

c: 0 ;lambda: 3.6e-06


/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in multiply


current_budget: 6250000
wintimes: 0
----------------
c: 30 ;lambda: 3.6e-06
current_budget: 3779472
wintimes: 93912
----------------
c: 60 ;lambda: 3.6e-06
current_budget: 1784726
wintimes: 126204
----------------
c: 90 ;lambda: 3.6e-06
current_budget: 821055
wintimes: 139201
----------------
c: 120 ;lambda: 3.6e-06
current_budget: 275072
wintimes: 145795
----------------
c: 150 ;lambda: 3.6e-06
lack of budget! -39
current_budget: -39
wintimes: 147857
----------------
c: 180 ;lambda: 3.6e-06
lack of budget! -42
current_budget: -42
wintimes: 144891
----------------
c: 210 ;lambda: 3.6e-06
lack of budget! -46
current_budget: -46
wintimes: 142885
----------------
c: 240 ;lambda: 3.6e-06
lack of budget! -1
current_budget: -1
wintimes: 141453
----------------
c: 270 ;lambda: 3.6e-06
lack of budget! -12
current_budget: -12
wintimes: 140456
----------------
c: 300 ;lambda: 3.6e-06
lack of budget! -10
current_budget: -10
wintimes: 139730
----------------
c: 330 ;lambda: 3.6e-06
lack of budget!

In [18]:
cs = np.arange(50,150,10)
lam = 3.6e-06
# maxClks=[]
# for lam in lams:
Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
fig = plt.figure()
print(max(Clks))
# maxClks.append(max(Clks))
plt.plot(cs,Clks)
mpld3.display(fig)
    
# print('maxmax:',max(maxClks))

c: 50 ;lambda: 3.6e-06
current_budget: 2294366
wintimes: 118521
----------------
c: 60 ;lambda: 3.6e-06
current_budget: 1784726
wintimes: 126204
----------------
c: 70 ;lambda: 3.6e-06
current_budget: 1391141
wintimes: 131759
----------------
c: 80 ;lambda: 3.6e-06
current_budget: 1082011
wintimes: 135888
----------------
c: 90 ;lambda: 3.6e-06
current_budget: 821055
wintimes: 139201
----------------
c: 100 ;lambda: 3.6e-06
current_budget: 608601
wintimes: 141841
----------------
c: 110 ;lambda: 3.6e-06
current_budget: 423540
wintimes: 144050
----------------
c: 120 ;lambda: 3.6e-06
current_budget: 275072
wintimes: 145795
----------------
c: 130 ;lambda: 3.6e-06
current_budget: 143359
wintimes: 147270
----------------
c: 140 ;lambda: 3.6e-06
current_budget: 24349
wintimes: 148555
----------------
164


In [19]:
cs = np.arange(120,150,1)
lam = 3.6e-06
# maxClks=[]
# for lam in lams:
Clks = [calculate_click(Train_df,payprice,c, lam, pCTR) for c in cs]
fig = plt.figure()
print(max(Clks))
# maxClks.append(max(Clks))
plt.plot(cs,Clks)
mpld3.display(fig)
    
# print('maxmax:',max(maxClks))

c: 120 ;lambda: 3.6e-06
current_budget: 275072
wintimes: 145795
----------------
c: 121 ;lambda: 3.6e-06
current_budget: 261676
wintimes: 145941
----------------
c: 122 ;lambda: 3.6e-06
current_budget: 248110
wintimes: 146101
----------------
c: 123 ;lambda: 3.6e-06
current_budget: 234940
wintimes: 146248
----------------
c: 124 ;lambda: 3.6e-06
current_budget: 221423
wintimes: 146398
----------------
c: 125 ;lambda: 3.6e-06
current_budget: 208504
wintimes: 146544
----------------
c: 126 ;lambda: 3.6e-06
current_budget: 193973
wintimes: 146714
----------------
c: 127 ;lambda: 3.6e-06
current_budget: 182857
wintimes: 146836
----------------
c: 128 ;lambda: 3.6e-06
current_budget: 169810
wintimes: 146982
----------------
c: 129 ;lambda: 3.6e-06
current_budget: 155274
wintimes: 147142
----------------
c: 130 ;lambda: 3.6e-06
current_budget: 143359
wintimes: 147270
----------------
c: 131 ;lambda: 3.6e-06
current_budget: 129600
wintimes: 147423
----------------
c: 132 ;lambda: 3.6e-06
curr

In [20]:
df_test = pd.read_csv("./test_result.csv")

In [21]:
c = 140
lam = 3.6e-06

In [22]:
calculate_click(Train_df,payprice,c, lam, pCTR)

c: 140 ;lambda: 3.6e-06
current_budget: 24349
wintimes: 148555
----------------


164

In [23]:
df_test.head()

bidprice  click_pred_per  click_pred_class
0  117.668617        0.000729                 0
1  287.101776        0.001779                 0
2   27.394619        0.000170                 0
3   83.446358        0.000517                 0
4   68.926285        0.000427                 0

In [24]:
testCTR = df_test['click_pred_per'].as_matrix()

In [25]:
bidprice_test = generate_bidprice(c, lam, testCTR)

In [26]:
df_test['bidprice'] = bidprice_test

In [27]:
df_test.head()

bidprice  click_pred_per  click_pred_class
0  111.489432        0.000729                 0
1  197.838492        0.001779                 0
2   30.939271        0.000170                 0
3   85.243569        0.000517                 0
4   72.603397        0.000427                 0

In [36]:
df_test.to_csv("ortb1-164",encoding='utf-8', index=False)

In [9]:
# #字典中的key值即为csv中列名
# dataframe = pd.DataFrame({'bidprice':bidprices})

#将DataFrame存储为csv,index表示是否显示行名，default=True
# dataframe.to_csv("bidprice10-40.csv",index=False,sep=',')

In [ ]:
# group_df = pd.read_csv('/Users/qrdai/web_project/we_data/train.csv')